# MOS Capacitor

## Capacitance-Voltage (C-V) Analysis

The MOS capacitor is the fundamental building block for understanding MOSFET operation. C-V measurements reveal important information about oxide quality and interface states.

**Learning Objectives:**
- Understand MOS capacitor operation regions
- Simulate C-V characteristics
- Analyze accumulation, depletion, and inversion
- Study flat-band voltage and oxide charges

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## Device Parameters Reference

The `describe()` function shows all available parameters for the MOS capacitor factory, including geometry, doping, physical models, and sweep options.

In [ ]:
from nanohubpadre import Simulation

# Show all available parameters for the MOS capacitor
Simulation.describe('mos_capacitor')

---

## 1. MOS Capacitor Physics

### 1.1 Structure

```
    Metal Gate
    ==========
    |  Oxide  | (SiO2)
    ==========
    |         |
    | Silicon | (P-type or N-type)
    |         |
    ==========
    Back Contact
```

### 1.2 Operation Regions (P-type substrate)

- **Accumulation** (Vg < 0): Holes accumulate at surface
- **Flat-band** (Vg = VFB): Bands are flat
- **Depletion** (VFB < Vg < VT): Surface depleted of holes
- **Inversion** (Vg > VT): Electrons form inversion layer

### 1.3 Key Parameters

- **Oxide capacitance**: $C_{ox} = \epsilon_{ox}/t_{ox}$

- **Flat-band voltage**: $V_{FB} = \phi_{ms} - Q_f/C_{ox}$

- **Threshold voltage**: $V_T = V_{FB} + 2\phi_F + Q_d/C_{ox}$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nanohubpadre import create_mos_capacitor

# Physical constants
q = 1.6e-19
eps_0 = 8.85e-14  # F/cm
eps_ox = 3.9 * eps_0
eps_si = 11.7 * eps_0
kT = 0.0259  # eV at 300K
ni = 1.5e10  # intrinsic carrier concentration

# MOS parameters
tox = 5e-7   # 5 nm oxide
Na = 1e17    # P-type substrate doping

Cox = eps_ox / tox
phi_F = kT * np.log(Na / ni)

print("MOS Capacitor Parameters:")
print("="*40)
print(f"Oxide thickness: {tox*1e7:.0f} nm")
print(f"Cox = {Cox*1e6:.2f} uF/cm^2")
print(f"phi_F = {phi_F:.3f} V")


---

## 2. Creating a MOS Capacitor Simulation

In [ ]:
# Create MOS capacitor
sim_moscap = create_mos_capacitor(
    # Geometry
    oxide_thickness=0.005,      # 5 nm oxide
    silicon_thickness=0.05,     # 50 nm silicon
    device_width=1.0,
    # Doping
    substrate_doping=1e17,
    substrate_type='p',
    # Gate
    gate_type='n_poly',
    # Oxide
    oxide_permittivity=3.9,
    # Models
    temperature=300,
    # Output
    log_bands_eq=True,
)

result = sim_moscap.run()
if result.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result.stderr}")


In [ ]:
# Visualize the MOS capacitor device structure
sim_moscap.device_schematic()

In [ ]:
# Plot equilibrium band diagram
sim_moscap.plot_band_diagram(title="MOS Capacitor - Equilibrium (Near Flat-band)")


---

## 3. C-V Characteristics

In [ ]:
# C-V simulation with Rappture default parameters
from nanohubpadre import create_mos_capacitor

sim_cv = create_mos_capacitor(
    # Geometry — Rappture defaults
    oxide_thickness=0.1,         # t_ox = 0.1 um (100 nm)
    ny_oxide=100,
    silicon_thickness=5.0,
    ny_silicon=200,
    # Doping — Rappture default Na = 1e15 /cm3
    substrate_doping=1e15,
    substrate_type='p',
    # Gate — Rappture default: n+ poly silicon
    gate_type='n_poly',
    oxide_permittivity=3.9,
    # Carrier lifetimes — 1 ns each
    taun0=1e-9,
    taup0=1e-9,
    temperature=300,
    # HF C-V (1 MHz)
    log_cv=True,
    cv_file='cv_hf',
    ac_frequency=1e6,
    # LF C-V (1 Hz)
    log_cv_lf=True,
    cv_lf_file='cv_lf',
    ac_frequency_lf=1.0,
    # Equilibrium band diagram + quasi-Fermi levels
    log_bands_eq=True,
    log_qf_eq=True,
    # Equilibrium carrier/potential/E-field profiles
    log_profiles_eq=True,
    # Gate voltage sweep — Rappture default: -3V to 5V, 100 steps
    vg_sweep=(-3.0, 5.0, 0.08),
)

result_cv = sim_cv.run()
if result_cv.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_cv.stderr}")


In [ ]:
# Band diagram at equilibrium (Ec, Ev, Efn, Efp)
sim_cv.plot_band_diagram(suffix="eq", title="Band Diagram at Equilibrium (Vg = 0)")

# Carrier concentrations at equilibrium
sim_cv.plot_carriers(suffix="eq", log_scale=True, title="Carrier Concentrations at Equilibrium")

# Electrostatic potential and electric field
sim_cv.plot_electrostatics(suffix="eq", title="Electrostatics at Equilibrium (Vg = 0)")

# C-V curves: HF and LF
sim_cv.plot_cv(title="MOS Capacitor C-V (HF + LF)")


In [ ]:
# Theoretical C-V curve and PADRE simulation comparison
def mos_cv(Vg, Na, tox, VFB=0):
    """Calculate MOS capacitor C-V curve"""
    Cox = eps_ox / tox
    phi_F = kT * np.log(Na / ni)
    
    C = np.zeros_like(Vg)
    
    for i, vg in enumerate(Vg):
        Vgb = vg - VFB
        
        if Vgb < 0:  # Accumulation
            C[i] = Cox
        elif Vgb < 2 * phi_F:  # Depletion
            # Simplified depletion capacitance
            Wd = np.sqrt(2 * eps_si * Vgb / (q * Na))
            Cd = eps_si / Wd
            C[i] = 1 / (1/Cox + 1/Cd)
        else:  # Strong inversion (high frequency)
            Wd_max = np.sqrt(4 * eps_si * phi_F / (q * Na))
            Cd_min = eps_si / Wd_max
            C[i] = 1 / (1/Cox + 1/Cd_min)  # HF: Cmin
    
    return C

Vg = np.linspace(-2, 2, 100)
C_hf = mos_cv(Vg, Na, tox)

fig = go.Figure()

# Theoretical curves
fig.add_trace(go.Scatter(x=Vg, y=C_hf / Cox * 100, mode="lines",
    line=dict(color="blue", width=2, dash="dash"), opacity=0.5,
    name="Theory: High Frequency"))

# Low frequency curve (inversion capacitance = Cox)
C_lf = mos_cv(Vg, Na, tox)
C_lf[Vg > 0.5] = Cox  # LF: inversion charge follows
fig.add_trace(go.Scatter(x=Vg, y=C_lf / Cox * 100, mode="lines",
    line=dict(color="red", width=2, dash="dash"), opacity=0.5,
    name="Theory: Low Frequency"))

# Try to get PADRE C-V data
try:
    cv_data = sim_cv.get_cv_data()
    if cv_data is not None:
        Vg_sim, C_sim = cv_data.get_cv_data()
        # Normalize to Cox
        Cox_val = eps_ox / (tox)
        fig.add_trace(go.Scatter(x=Vg_sim, y=C_sim / Cox_val * 100, mode="lines",
            line=dict(color="green", width=2), name="PADRE Simulation"))
        print("PADRE C-V data extracted successfully!")
except Exception as e:
    print(f"Note: C-V data extraction may require additional output parsing: {e}")
    print("Using theoretical curves for comparison")

fig.update_layout(template="plotly_white", width=1000, height=450,
    title_text="MOS Capacitor C-V Characteristics")
fig.update_xaxes(title_text="Gate Voltage Vg (V)", range=[-2, 2])
fig.update_yaxes(title_text="Capacitance C/Cox (%)", range=[0, 110])

# Add region labels
fig.add_annotation(text="Accumulation", x=-1.5, y=95, showarrow=False, font=dict(size=11))
fig.add_annotation(text="Depletion", x=0, y=60, showarrow=False, font=dict(size=11))
fig.add_annotation(text="Inversion", x=1.5, y=40, showarrow=False, font=dict(size=11))

fig.show()

# Show theoretical parameters
phi_F_val = kT * np.log(Na / ni)
Wd_max = np.sqrt(4 * eps_si * phi_F_val / (q * Na)) * 1e4  # um
Cd_min = eps_si / (Wd_max * 1e-4)
Cmin_Cox = 1 / (1 + Cox / Cd_min)

print("\nTheoretical Parameters:")
print("="*40)
print(f"Surface potential at inversion: 2*phi_F = {2*phi_F_val:.3f} V")
print(f"Maximum depletion width: Wd_max = {Wd_max*1e3:.1f} nm")
print(f"Cmin/Cox = {Cmin_Cox*100:.1f}%")


---

## 4. Effect of Doping

In [ ]:
# Compare different substrate dopings with PADRE simulations
doping_levels = [1e16, 1e17, 1e18]  # cm^-3

doping_simulations = {}
for Na_val in doping_levels:
    sim = create_mos_capacitor(
        oxide_thickness=0.005,
        substrate_doping=Na_val,
        substrate_type='p',
        log_cv=True,
        log_bands_eq=True,
        vg_sweep=(-2.0, 2.0, 0.1),
    )
    result = sim.run()
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed for Na={Na_val:.0e}:\n{result.stderr}")
    doping_simulations[Na_val] = sim

# Plot C-V for each doping level
for Na_val, sim in doping_simulations.items():
    sim.plot_cv(title=f"C-V — Na = {Na_val:.0e} cm⁻³")


---

## 5. Effect of Oxide Thickness

In [ ]:
# Compare different oxide thicknesses with PADRE simulations
tox_values = [0.002, 0.005, 0.010]  # 2, 5, 10 nm (in um)

tox_simulations = {}
for tox_um in tox_values:
    sim = create_mos_capacitor(
        oxide_thickness=tox_um,
        substrate_doping=1e17,
        substrate_type='p',
        log_cv=True,
        log_bands_eq=True,
        vg_sweep=(-2.0, 2.0, 0.1),
    )
    result = sim.run()
    if result.returncode != 0:
        raise RuntimeError(f"Simulation failed for tox={tox_um*1000:.0f} nm:\n{result.stderr}")
    tox_simulations[tox_um] = sim

# Band diagrams for different oxide thicknesses
for tox_um, sim in tox_simulations.items():
    sim.plot_band_diagram(title=f"MOS Capacitor — tox = {tox_um*1000:.0f} nm")


---

## 6. Band Diagrams in Different Regions

In [ ]:
# Simulate band diagrams at different gate voltages
sim_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    substrate_doping=1e17,
    substrate_type='p',
    log_bands_eq=True,
    log_bands_bias=True,
    log_cv=True,
    vg_sweep=(-1.0, 1.5, 0.5),
)

result_bands = sim_bands.run()
if result_bands.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_bands.stderr}")

# Equilibrium and bias band diagrams
sim_bands.plot_band_diagram(title="MOS Capacitor — Band Structure")


---

## 7. Complete Simulation Example

In [ ]:
# Complete MOS capacitor characterization
sim_complete = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.05,
    device_width=1.0,
    nx=3,
    ny_oxide=15,
    ny_silicon=35,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    oxide_permittivity=3.9,
    oxide_qf=0,
    temperature=300,
    conmob=True,
    fldmob=True,
    log_cv=True,
    cv_file="moscap_cv",
    log_bands_eq=True,
    vg_sweep=(-2.5, 2.5, 0.05),
    ac_frequency=1e6,
)

result_complete = sim_complete.run()
if result_complete.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_complete.stderr}")

sim_complete.plot_band_diagram(title="Complete MOS Capacitor — Equilibrium Band Diagram")
sim_complete.plot_cv(title="Complete MOS Capacitor C-V")


---

## 8. Double-Gate MOS Capacitor

In a **double-gate** configuration a second gate oxide and gate electrode replace the ohmic back contact.  Both gates couple to the silicon body simultaneously, giving independent control of the top and bottom surfaces.  This geometry is a building block for fully-depleted SOI (FD-SOI) devices and FinFETs.

```
    Gate 1 (top)
    ============
    |  Oxide 1 |  ← top SiO2
    ============
    |  Silicon |  ← thin body (P or N)
    ============
    |  Oxide 2 |  ← bottom SiO2
    ============
    Gate 2 (bottom)
```

Key differences from the single-gate MOS cap:
- **Two coupling oxides** — can use the same or different thicknesses.
- **Electrode 1** = top gate (swept), **Electrode 2** = bottom gate (fixed bias or also swept).
- The body is depleted from both sides simultaneously, which halves the required gate swing.

Use `gate_config="double"` in `create_mos_capacitor` to enable this mode.

In [ ]:
# 8.1 Single-gate vs double-gate: schematic comparison
from nanohubpadre import create_mos_capacitor
from IPython.display import display

# Single-gate (default)
sim_sg = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='single',
)
print('Single-gate structure:')
display(sim_sg.device_schematic())

# Double-gate
sim_dg_sch = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.005,
    back_gate_type='n_poly',
)
print('Double-gate structure:')
display(sim_dg_sch.device_schematic())

In [ ]:
# 8.2 Double-gate MOS capacitor: structure and C-V
from IPython.display import display

sim_dg = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.005,
    back_gate_type='n_poly',
    log_cv=True,
    cv_file='cv_dg',
    log_bands_eq=True,
    vg_sweep=(-2.0, 2.0, 0.1),
    ac_frequency=1e6,
)

display(sim_dg.device_schematic())

result_dg = sim_dg.run()
if result_dg.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_dg.stderr}")

sim_dg.plot_band_diagram(title="Double-Gate MOS Cap — Equilibrium")
sim_dg.plot_cv(title="Double-Gate C-V")


In [ ]:
# 8.3 Compare equilibrium band diagrams: single-gate vs double-gate
sim_sg_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='single',
    log_bands_eq=True,
)
result_sg = sim_sg_bands.run()
if result_sg.returncode != 0:
    raise RuntimeError(f"Single-gate simulation failed:\n{result_sg.stderr}")

sim_dg_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.005,
    back_gate_type='n_poly',
    log_bands_eq=True,
)
result_dg = sim_dg_bands.run()
if result_dg.returncode != 0:
    raise RuntimeError(f"Double-gate simulation failed:\n{result_dg.stderr}")

sim_sg_bands.plot_band_diagram(title='Single-Gate MOS Cap — Equilibrium')
sim_dg_bands.plot_band_diagram(title='Double-Gate MOS Cap — Equilibrium')


In [ ]:
# 8.4 Asymmetric double-gate: different top/bottom oxide thicknesses
sim_asym = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.020,
    back_gate_type='n_poly',
    log_bands_eq=True,
)
display(sim_asym.device_schematic())

result_asym = sim_asym.run()
if result_asym.returncode != 0:
    raise RuntimeError(f"Simulation failed:\n{result_asym.stderr}")

sim_asym.plot_band_diagram(title='Asymmetric Double-Gate — Equilibrium')


---

## Summary

In this notebook, you learned:

1. **MOS Structure**: Metal-Oxide-Semiconductor stack
2. **Operating Regions**: Accumulation, depletion, inversion
3. **C-V Analysis**: High vs low frequency behavior
4. **Doping Effects**: Higher doping -> higher Cmin
5. **Oxide Thickness**: Thinner oxide -> higher Cox
6. **Double-Gate**: Symmetric gate-oxide-Si-oxide-gate stack; depletes body from both sides

**Key Equations:**
- $C_{ox} = \epsilon_{ox}/t_{ox}$
- $C_{min}/C_{ox}$ depends on $W_d^{max}$ and doping
- Double-gate: $C_{ox,eff}^{-1} = C_{ox,top}^{-1} + C_{ox,bot}^{-1}$ in series

**Next**: [08 - MESFET](08_MESFET.ipynb) - Metal-semiconductor FETs